# EXAMPLE 7
## Explaining the experiment
In this example, we are investigating the bitstream of an FPGA. Its size is 955,760x32 = 30,584,320, containing 25,479,936 bits of configuration memory, 131,072 flip-flops, and 4,973,312 bits of BRAM. A golden file following the manufacturer's rules was uploaded on the FPGA and then irradiated in static mode in an only round. Later, the content was downloaded and compared with the initial file. Although we can split the raw bitstream in three sets thanks to some reverse engineering, we have preferred not to do that in order to force the tool to do its best.

## Loading packages
The very first thing we must do is to load the packages required to load files (_DelimitedFiles_) as well as the LELAPE module. I suppose you have installed both. Load is done with:

In [ ]:
### Defining the path to LELAPE
push!(LOAD_PATH, "PATH_TO_FOLDER_WITH_LELAPE.jl"); # <-- ADAPT THIS INSTRUCTION TO YOUR COMPUTER!
## If you are a Windows user, remember that subfolders are indicated with \\ or /, NEVER with a simple backslash.

In [ ]:
using DelimitedFiles, LELAPE

## Defining variables
Previous paragraph allows us to define several variables for checking the tests:

* Word width : 32 bits
* Memory size in words: 800344.
* In FPGAs, it seems more likely to succeed the POS operation.
* Tests were static. No information about cycles is necessary.

Ok, let us use this information to set these variables:

In [ ]:
LA = 955760 # Memory size in words
WordWidth = 32 # Selfexplaining.
Operation = "POS" # Only "XOR" or "POS" are allowed.
KeepCycles = false # This is a Bool variable and only true false are accepted.

## Loading data
Results are stored in three different files following the required format: 
* CSV files 
* Every row is formed as WORD ADDRESS, READ VALUE, PATTERN, CYCLE.
Besides, the first row contains column heading (must be skipped), separators are commas and EOL character is the standard. 

We will use the _readdlm_ function provided by the _DelimitedFiles_ package to load the first CSV file and to store everything in the new variable, DATA. Finally, it is important to indicate that DATA must be an array of UInt32 numbers. 

In [ ]:
DATA1 = readdlm("ExampleFPGA11.csv", ',', UInt32, '\n', skipstart=1)

Good!! If you have correctly proceeded, _ExampleFPGA11_returned a 1325x4 unsigned integer matrix is loaded. Now, let us analyze the DATA1.

## Looking for MBUs
This analyisis is quite simple. We will call the _CheckMBUs_ function, which returns the MBUs present in DATA.Input arguments are the second and third columns, and the wordwidth.

This function returns two vectors. The first one indicates in position _k_ the number of bitflips observed in the _kth_ word. The second one is a vector of vectors and contains more detailed information: not only the number of bitflips per word but the position of the flipped bit (0 = LSB, WordWidth-1 = MSB). 

In [ ]:
MBUSize, MBU_bit_pos = CheckMBUs(DATA1[:,2], DATA1[:,3], WordWidth)

The following loop will show how many MBUs per number of flipped bits were observed:

In [ ]:
TNFB = 0 # Total number of bitflpis
for size = 1: WordWidth
    NMBUs = length(findall(MBUSize.==size))
    TNFB += NMBUs*size
    NMBUs!=0 ? println("$size-bit MBUs: ", NMBUs) : nothing
end
println("\n$TNFB bitflips")

Interesting! There are some MBUs but... are they real or just the accumulation of SBUs. Let us perform an analysis of the presence of false 2- and 3-bit MBUs.

In [ ]:
NF2BMBU = NF2BitMCUs(TNFB, LA, "MBU", WordWidth, WordWidth, true)
println("You expected $NF2BMBU and got $(length(findall(MBUSize.==2))). What do you believe?")


## Looking for MCUs
As modern memories are interleaved, it is not worth investigating MBUs but MCUs. Now, the system will combine addresses in all the possible pairs and operate them to create a DV set. If there were no MCUs, their characteristics are known. 

In particular, we can state that if the expected number of elements repeated _k_ times in this set is lower than a very low positive number, it is impossible to observe this number of repetitions unless the Only SBU assumption fails. We will define this threshold as 0.001 (default, 0.05). 


Although without a solid theoretical background, it seems that using pseudoaddress instead of word address provides better results.

Some experiments seem to show that if an element with very few number of 1s in binary format is too often repeated, it is indicative of the presence of MCUs. This is the Trace Rule and, in our analyisis, we want to keep all those too often repeated elements such that contain 2 ones or less in binary format.

Finally, perhaps we know that MCUs will not very large. For example, we may guess that MCUs with more than 20 bitflips are totally rejected. Therefore, to help the software and to avoid running out of memory, we will say the program _"Don't be silly and do not expect events larger than 20!!"_ If somehow this idea was wrong, we can change this value again and repeat the calculations.

__NOTE__: _In this experiments, it occurs that there will be very, very large events. Therefore, we are going to suppose that events can go up 500._

In [ ]:
ϵ = 0.001   # If the expected number of elements repeated k times is lower than ϵ, 
            # we can afirm that this is virtually impossible.
UsePseudoAddress = true
TraceRuleLength = 2
LargestMCUSize = 500

Time to test!!! We will call the function. Deppending on the set size or even if this is your first test, it will take you more or less time (Don't get up from your chair, though!!!!)

The following instruction will look for:
1. Values that pass the self-consistency test (C1_SCY)
2. Values found after inspecting MCUs derived from self-consistency-test (C1_MCU).
3. Values with less than or equal to _TraceRuleLength_ 1s in binary format that appear too often in the DV set (C1_TRC).
4. Values that, after combining in pairs the union of all the previous three sets and applying the operation and that appear too many times within the DV set (C1_SHF).

The first column of each matrix are the possible values and the second one the times it appeared.

In [ ]:
C1_SCY, C1_MCU, C1_TRC, C1_SHF = DetectAnomalies_FullCheck(DATA1, WordWidth, LA, Operation, TraceRuleLength, UsePseudoAddress, KeepCycles, ϵ, LargestMCUSize)

Perhaps these matrices are hard to read since, for efficiency, they were returned in UInt32 format, even the number of occurrences!!! Execute the following instrucction for a better comprehension.

In [ ]:
println("Elements appearing more than expected and passing the Self-Consistency test:\n")
for index in 1:length(C1_SCY[:, 1])
    println("Value: 0x", string(C1_SCY[index, 1], base=16, pad = 6), " (", Int(C1_SCY[index, 1]), ") --> ", Int(C1_SCY[index, 2]),".")
end

UsePseudoAddress ? L = LA*WordWidth : L = LA

print("\nOnly up to ", MaxExpectedRepetitions(NPairs(DATA1, UsePseudoAddress, WordWidth, KeepCycles), L, Operation, ϵ)-1, " repetitions are explained by randomness.")

In the case of using _ExampleFPGA11.csv_, you will find 4 values, 3231, 3232, 6464, and 9696. The first two values were already found in Example 5. The latter two values are just 3232×2 and 3232×3. Now, let us summaryze sets and start to group bitflips.

In [ ]:
C1_All = [C1_SCY; C1_MCU; C1_TRC; C1_SHF]

## Grouping bitflips
Now, we have discovered those values relating pairs of pseudoaddresses. Now, let us go to group events in DATA. 

The first step consists in labeling all the pseudoaddresses and grouping their assigned indexes to a matrix containing information for the possible MCUs. It is an intermediate step and is done with the instruction _MCU_Indexes_ with the required and already defined parameters. 

In [ ]:
Labeled_addresses = MCU_Indexes(DATA1, Operation, C1_All[:, 1], UsePseudoAddress, WordWidth)

You should have got a 198×128 matrix so there are 198 multiple events, with at least one with 128 bits.  

Now, we will classify addresses with _Classify_Addresses_in_MCU()_, using the previous matrix, _Labeled_addresses_. 

In [ ]:
Events = Classify_Addresses_in_MCU(DATA1, Labeled_addresses, UsePseudoAddress, WordWidth)

---------------------
Difficult to read, isn't it? The following instruction makes the content more readable:

In [ ]:
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    if NMCUs != 0
        println("Pseudoaddresses involved in $(length(Events)-k+1)-bit MCUs ($NMCUs events):")
        for row = 1:NMCUs
            for bit = 1:length(Events)-k+1
                print("0x", string(Events[k][row, bit], base=16, pad = 6), )
            
                bit != length(Events)-k+1 ? print(", ") : print("\n")

            end
        end
        println()
    end
end


Let us summaryze the number of occurrences.

In [ ]:
##################
println("Multiplicity\tOccurrences")
println("------------\t-----------")
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    if NMCUs != 0
        println("$(length(Events)-k+1)\t\t$NMCUs")
    end
end

## Analysis completed but...
You can find other data sets in this folder. However, there are more and more events and it is likely that your computer be commited. Try at your risk if you wish or go on to the second part of the notebook.

# Second part 
Fortunately, this FPGA was already tested in Example 5, so we know the values linking bitflips in the bitstream:

* 1
* 2
* 3230
* 3231
* 3232
* 3233
* 3234

Also, other values were discovered with _ExampleFPGA11.csv_:

* 6464
* 9696

Therefore, we can just define the anomalies and verify all the sets. Let us go:

In [ ]:
Anomalies = convert.(UInt32, [1; 2; 3230:3234; 6464; 9696]); # The anomalies as a UInt32 Vector

Let us reload the CSV file. Change it if you wish:

In [ ]:
DATA1 = readdlm("ExampleFPGA11.csv", ',', UInt32, '\n', skipstart=1);

Recalculate the event shapes...

In [ ]:
Labeled_addresses = MCU_Indexes(DATA1, Operation, Anomalies, UsePseudoAddress, WordWidth)

Let us show the MCUs with their corresponding addreseses:


In [ ]:
Events = Classify_Addresses_in_MCU(DATA1, Labeled_addresses, UsePseudoAddress, WordWidth)
####
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    if NMCUs != 0
        println("Pseudoaddresses involved in $(length(Events)-k+1)-bit MCUs ($NMCUs events):")
        for row = 1:NMCUs
            for bit = 1:length(Events)-k+1
                print("0x", string(Events[k][row, bit], base=16, pad = 6), )
            
                bit != length(Events)-k+1 ? print(", ") : print("\n")

            end
        end
        println()
    end
end


Or, in short:

In [ ]:
##################
println("Multiplicity\tOccurrences")
println("------------\t-----------")
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    if NMCUs != 0
        println("$(length(Events)-k+1)\t\t$NMCUs")
    end
end

The use of a larger set of anomalies has improved results in the case of _ExampleFPGA11.csv_, although the misterious large event has still 128 bits, a power of 2.

Now, you can just analyze all the other sets of events. Good luck!

# More information
These results were the basis for this work: J. C. Fabero, G. Korkian, F. J. Franco, H. Mecha, M. Letiche and J. A. Clemente, _"Thermal Neutron-induced SEUs on a COTS 28-nm SRAM-based FPGA under Different Incident Angles,_" 2021 IEEE 22nd Latin American Test Symposium (LATS), 2021, pp. 1-6, doi: 10.1109/LATS53581.2021.9651879.

Go there for further details.